In [ ]:
  !ngrok config add-authtoken 32nSDd4hW3onntDaAsffc8VK1ih_4RGedvBLdeJ5gr6ai67eG

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
# ========================================
# 1. Instalación de dependencias
# ========================================
!pip install dash==2.16.1 jupyter-dash paho-mqtt --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 52.9 MB/s eta 0:00:00


In [ ]:
# ========================================
# 2. Importaciones
# ========================================
import json
import pandas as pd
import paho.mqtt.client as mqtt
from dash import Dash, dcc, html, dash_table
from dash.dependencies import Output, Input
from jupyter_dash import JupyterDash

# ========================================
# 3. Variables globales
# ========================================
broker = "45abc320943e4d708930117ef02e02c5.s1.eu.hivemq.cloud"
port = 8883
topic = "esp8266/mpu6050"
username = "TU_USER"       # 🔹 pon tu usuario HiveMQ Cloud
password = "TU_PASSWORD"   # 🔹 pon tu password HiveMQ Cloud

# DataFrame inicial
df = pd.DataFrame(columns=["fecha", "ax", "ay", "az", "gx", "gy", "gz", "temp"])


In [ ]:
# ========================================
# 4. Cliente MQTT
# ========================================
def on_connect(client, userdata, flags, rc, properties=None):
    print("🔌 MQTT conectado, rc =", rc)
    client.subscribe(topic)

def on_message(client, userdata, msg):
    global df
    try:
        data = json.loads(msg.payload.decode())
        new_row = pd.DataFrame([{
            "fecha": data.get("fecha"),
            "ax": data.get("ax"),
            "ay": data.get("ay"),
            "az": data.get("az"),
            "gx": data.get("gx"),
            "gy": data.get("gy"),
            "gz": data.get("gz"),
            "temp": data.get("temp"),
        }])
        df = pd.concat([df, new_row], ignore_index=True).tail(100)  # guarda últimas 100 filas
    except Exception as e:
        print("⚠️ Error procesando mensaje:", e)

client = mqtt.Client(mqtt.CallbackAPIVersion.VERSION2)
client.username_pw_set(username, password)
client.on_connect = on_connect
client.on_message = on_message
client.tls_set()  # obligatorio HiveMQ Cloud
client.connect(broker, port)
client.loop_start()


<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>

In [ ]:
# ========================================
# 5. Dashboard con Dash
# ========================================
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H2("📡 Datos en vivo del ESP8266 + MPU6050"),
    dash_table.DataTable(
        id="live-table",
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict("records"),
        style_table={"height": "400px", "overflowY": "auto"},
        style_cell={"textAlign": "center", "padding": "5px"},
        page_size=20
    ),
    dcc.Interval(
        id="interval-component",
        interval=1000,  # cada 1 segundo actualiza
        n_intervals=0
    )
])

@app.callback(
    Output("live-table", "data"),
    Input("interval-component", "n_intervals")
)
def update_table(n):
    global df
    return df.to_dict("records")

print("🔁 Iniciando dashboard (inline)...")
app.run_server(mode="inline", port=8051)


🔁 Iniciando dashboard (inline)...


/usr/local/lib/python3.12/dist-packages/dash/dash.py:550: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# ========================================
# Dashboard con Tabla + Gráfica en Tiempo Real
# ========================================
import plotly.express as px

app = JupyterDash(__name__)

app.layout = html.Div([
    html.H2("📡 Datos en vivo del ESP8266 + MPU6050"),

    # Tabla
    dash_table.DataTable(
        id="live-table",
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict("records"),
        style_table={"height": "300px", "overflowY": "auto"},
        style_cell={"textAlign": "center", "padding": "5px"},
        page_size=10
    ),

    html.H3("📊 Gráfica en tiempo real (ax, ay, az, gx, gy, gz)"),
    dcc.Graph(id="live-graph"),

    dcc.Interval(
        id="interval-component",
        interval=1000,  # cada 1 segundo refresca
        n_intervals=0
    )
])

# Callback: actualizar tabla
@app.callback(
    Output("live-table", "data"),
    Input("interval-component", "n_intervals")
)
def update_table(n):
    global df
    return df.to_dict("records")

# Callback: actualizar gráfica
@app.callback(
    Output("live-graph", "figure"),
    Input("interval-component", "n_intervals")
)
def update_graph(n):
    global df
    if df.empty:
        return px.line(title="Esperando datos...")

    # Seleccionamos solo las últimas 100 filas
    dff = df.tail(100)

    # Gráfica de múltiples líneas
    fig = px.line(
        dff,
        x="fecha",
        y=["ax", "ay", "az", "gx", "gy", "gz"],
        title="Acelerómetro y Giroscopio en tiempo real",
    )
    fig.update_layout(
        xaxis_title="Tiempo",
        yaxis_title="Valor",
        legend_title="Ejes",
        template="plotly_dark"
    )
    return fig

print("🔁 Iniciando dashboard (inline)...")
app.run_server(mode="inline", port=8052)


🔁 Iniciando dashboard (inline)...


/usr/local/lib/python3.12/dist-packages/dash/dash.py:550: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>